<a href="https://colab.research.google.com/github/asrenninger/wrangling/blob/master/python/ndvi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [3]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Px6SfdmySbeznziX8dRd2lWvg5WwSUIvh0v4Q9lQUMc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7E15zWDDulCOH6LAv8rbjg0xwk2L6OGyst1b2OgELi_9yOQCORy7Y

Successfully saved authorization token.


In [4]:
import folium

print(folium.__version__)

0.8.3


In [70]:
def maskcloud(img):
  QA60 = img.select(['QA60'])
  return img.updateMask(QA60.lt(0.1))

def addNDVI(img):
  return img.addBands(img.normalizedDifference(['B8', 'B4']))

def addNDWI(img):
  return img.addBands(img.normalizedDifference(['B8', 'B11']))

In [87]:
# sentinel imagery
sen2 = ee.ImageCollection('COPERNICUS/S2').map(maskcloud).filterDate('2020-01-01', '2020-12-31')
srtm = ee.Image('USGS/SRTMGL1_003')

green = sen2.map(addNDVI)
water = sen2.map(addNDWI)

green = green.qualityMosaic('nd')
water = water.qualityMosaic('nd')

# map it
map = folium.Map(location=[39.95375402879083, -75.1631614349256], zoom_start=15)

mapid = green.getMapId({'bands': 'nd', 'min': 0, 'max': 0.5})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDVI',
  ).add_to(map)

mapid = water.getMapId({'bands': 'nd', 'min': 0, 'max': 1})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDWI',
  ).add_to(map)

map.add_child(folium.LayerControl())
map


In [88]:
polys = ee.FeatureCollection('users/asrenninger/boxes')

polim = ee.Image(0).byte().paint(polys, 1)
polim = polim.updateMask(polim)

mapid = polim.getMapId({'palette': ['navy'], 'opacity': 0.5})
map = folium.Map(location=[37.664002801999125, -121.98408112276452], tiles="Stamen Toner", zoom_start=10)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='polygons',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

In [76]:
from google.colab import files
map.save("aoi_map.html")
files.download('aoi_map.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [85]:
co_day = ee.Image("users/asrenninger/conus_day")
hi_day = ee.Image("users/asrenninger/hi_day")
ak_day = ee.Image("users/asrenninger/ak_day")

combined = ee.ImageCollection.fromImages([co_day, hi_day, ak_day])
day = combined.qualityMosaic('b1')

co_night = ee.Image("users/asrenninger/conus_night")
hi_night = ee.Image("users/asrenninger/hi_night")
ak_night = ee.Image("users/asrenninger/ak_night")

combined = ee.ImageCollection.fromImages([co_night, hi_night, ak_night])
night = combined.qualityMosaic('b1')

map = folium.Map(location=[39.95375402879083, -75.1631614349256], zoom_start=15)

mapid = day.getMapId({'bands': 'b1', 'min': 0, 'max': 500})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Day',
  ).add_to(map)

mapid = night.getMapId({'bands': 'b1', 'min': 0, 'max': 500})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Night',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

In [108]:
ndvi = green.select(['nd'], ['ndvi'])
ndwi = water.select(['nd'], ['ndwi'])

indices = ndvi.addBands(ndwi)

In [109]:
lsd = day.select(['b1'], ['day'])
lsn = night.select(['b1'], ['night'])

density = lsd.addBands(lsn)

In [110]:
reducers = ee.Reducer.mean().combine(**{
  'reducer2': ee.Reducer.stdDev(),
  'sharedInputs': True
});

indices_reduced = indices.reduceRegions(**{
  'collection': polys,
  'reducer': reducers,
  'scale': 15,
});

density_reduced = density.reduceRegions(**{
  'collection': polys,
  'reducer': reducers,
  'scale': 15,
});

In [111]:
ee.batch.Export.table.toDrive(**{
  'collection': indices_reduced,
  'description': 'indices_reduced',
  'fileFormat': 'CSV'
}).start()

ee.batch.Export.table.toDrive(**{
  'collection': density_reduced,
  'description': 'density_reduced',
  'fileFormat': 'CSV'
}).start()